In [1]:
import torch

torch.__version__, torch.cuda.is_available()

/home/juan/Desktop/CUDA100/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


('2.9.0+cu128', True)

In [2]:
a = torch.tensor([1.0, 2.0, 3.0, 4.0])
b = torch.tensor([5.0, 6.0, 7.0, 8.0])

c = a + b

assert (c == torch.tensor([6.0, 8.0, 10.0, 12.0])).all()

In [3]:
%%time

N = 1000000000

a = torch.randn(N, device='cuda')
b = torch.randn(N, device='cuda')

c = a + b


CPU times: user 19.6 ms, sys: 125 ms, total: 144 ms
Wall time: 149 ms


In [4]:
%load_ext wurlitzer

In [5]:

import torch
from torch.utils.cpp_extension import load_inline

In [6]:

def load_cuda(cuda_src, cpp_src, funcs, opt=False, verbose=False):
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs,
                       extra_cuda_cflags=["-O2"] if opt else [], verbose=verbose, name="inline_ext")

In [7]:
cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)

inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a + b - 1) / b;}
'''

In [8]:
cuda_src = cuda_begin + r'''
__global__ void vectorAddKernel(float *out, float *a, float *b, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        out[i] = a[i] + b[i];
    }
}

torch::Tensor vectorAdd(torch::Tensor a, torch::Tensor b) {
    CHECK_INPUT(a); CHECK_INPUT(b);
    int n = a.size(0);
    auto output = torch::empty(n, a.options());
    int threads = 256;
    vectorAddKernel<<<cdiv(n, threads), threads>>>(output.data_ptr<float>(), a.data_ptr<float>(), b.data_ptr<float>(), n);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}'''

In [9]:
cpp_src = "torch::Tensor vectorAdd(torch::Tensor a, torch::Tensor b);"

In [10]:
module = load_cuda(cuda_src, cpp_src, ['vectorAdd'], verbose=True)


In [11]:
dir(module)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'vectorAdd']

In [12]:
%%time

N = 1000000000

a = torch.randn(N, device='cuda')
b = torch.randn(N, device='cuda')

c = module.vectorAdd(a, b)

CPU times: user 7.99 ms, sys: 37.9 ms, total: 45.9 ms
Wall time: 45.1 ms


In [13]:
cuda_src = cuda_begin + r'''
__global__ void vectorAddImprovedKernel(const float* __restrict__ A,
                           const float* __restrict__ B,
                           float* __restrict__ C, int N) {
    // Grid-stride loop: start at this thread's index, then stride by total threads
    for (int i = blockIdx.x * blockDim.x + threadIdx.x;
         i < N;
         i += blockDim.x * gridDim.x) {
        C[i] = A[i] + B[i];
    }
}

torch::Tensor vectorAddImproved(torch::Tensor a, torch::Tensor b) {
    CHECK_INPUT(a); CHECK_INPUT(b);
    int n = a.size(0);
    auto output = torch::empty(n, a.options());
    int threads = (n < 128) ? 32 : 128;
    int blocks = cdiv(n, threads);
    if (blocks == 0) blocks = 1;
    vectorAddImprovedKernel<<<blocks, threads>>>(a.data_ptr<float>(), b.data_ptr<float>(), output.data_ptr<float>(), n);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}'''

In [14]:
cpp_src = "torch::Tensor vectorAddImproved(torch::Tensor a, torch::Tensor b);"

In [15]:
module = load_cuda(cuda_src, cpp_src, ['vectorAddImproved'], verbose=True)

In [16]:
dir(module)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'vectorAddImproved']

In [17]:
%%time

N = 1000000000

a = torch.randn(N, device='cuda')
b = torch.randn(N, device='cuda')

c = module.vectorAddImproved(a, b)

CPU times: user 10.2 ms, sys: 21.8 ms, total: 32 ms
Wall time: 31.3 ms
